In [1]:
import dlib

print(dlib.cuda.get_num_devices())

import face_recognition
from PIL import Image
import os
import cv2
import numpy as np

In [9]:
image = face_recognition.load_image_file(r'C:\Users\14640\Pictures\unknown_pictures\WIN_20211019_20_07_41_Pro.jpg')
face_locations = face_recognition.face_locations(image)

# face_locations is now an array listing the co-ordinates of each face!
print(face_locations)

[(554, 1221, 1016, 759)]


In [10]:
# Load the jpg file into a numpy array
image = face_recognition.load_image_file(r'C:\Users\14640\Pictures\Camera Roll\刘青沅.jpg')

# Find all the faces in the image using the default HOG-based model.
# This method is fairly accurate, but not as accurate as the CNN model and not GPU accelerated.
# See also: find_faces_in_picture_cnn.py
face_locations = face_recognition.face_locations(image)

print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

I found 1 face(s) in this photograph.
A face is located at pixel location Top: 418, Left: 803, Bottom: 804, Right: 1188


In [4]:
import cv2
import numpy as np

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Load a sample picture and learn how to recognize it.
liuqingyuan_image = face_recognition.load_image_file(r"C:\Users\14640\Pictures\known_pictures\刘青沅.jpg")
liuqingyuan_face_encoding = face_recognition.face_encodings(liuqingyuan_image)[0]

# # Load a second sample picture and learn how to recognize it.
# biden_image = face_recognition.load_image_file("biden.jpg")
# biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    liuqingyuan_face_encoding,
#     biden_face_encoding
]
known_face_names = [
    "Liu Qingyuan",
#     "Joe Biden"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

'刘青沅.jpg'

In [8]:
os.path.splitext('刘青沅.jpg')[0]

'刘青沅'

In [15]:
known_img = face_recognition.load_image_file('C:/Users/14640/Pictures/known_pictures/路静琛.jpg')

In [22]:
known_face_encoding = face_recognition.face_encodings(known_img)

In [23]:
known_face_encoding != []

False

In [20]:
known_img[0]

array([[182, 186, 187],
       [182, 186, 187],
       [182, 186, 187],
       ...,
       [ 95, 114,  95],
       [ 99, 118,  98],
       [101, 119,  97]], dtype=uint8)

In [2]:
def add_alpha_channel(img):
    """ 为jpg图像添加alpha通道 """
 
    b_channel, g_channel, r_channel = cv2.split(img) # 剥离jpg图像通道
    alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 255 # 创建Alpha通道
 
    img_new = cv2.merge((b_channel, g_channel, r_channel, alpha_channel)) # 融合通道
    return img_new

In [3]:
def merge_img(jpg_img, png_img, y1, y2, x1, x2):
    """ 将png透明图像与jpg图像叠加 
        y1,y2,x1,x2为叠加位置坐标值
    """
    
    # 判断jpg图像是否已经为4通道
    if jpg_img.shape[2] == 3:
        jpg_img = add_alpha_channel(jpg_img)
    else:
        raise Exception('jpg图像不为3通道')
        
    # 判断png图像是否已经为4通道
    if png_img.shape[2] != 4:
        raise Exception('png图像不为4通道')
    
    '''
    当叠加图像时，可能因为叠加位置设置不当，导致png图像的边界超过背景jpg图像，而程序报错
    这里设定一系列叠加位置的限制，可以满足png图像超出jpg图像范围时，依然可以正常叠加
    '''
    yy1 = 0
    yy2 = png_img.shape[0]
    xx1 = 0
    xx2 = png_img.shape[1]
 
    if x1 < 0:
        xx1 = -x1
        x1 = 0
    if y1 < 0:
        yy1 = - y1
        y1 = 0
    if x2 > jpg_img.shape[1]:
        xx2 = png_img.shape[1] - (x2 - jpg_img.shape[1])
        x2 = jpg_img.shape[1]
    if y2 > jpg_img.shape[0]:
        yy2 = png_img.shape[0] - (y2 - jpg_img.shape[0])
        y2 = jpg_img.shape[0]
 
    # 获取要覆盖图像的alpha值，将像素值除以255，使值保持在0-1之间
    alpha_png = png_img[yy1:yy2,xx1:xx2,3] / 255.0
    alpha_jpg = 1 - alpha_png
    
    # 开始叠加
    for c in range(0,3):
        jpg_img[y1:y2, x1:x2, c] = ((alpha_jpg*jpg_img[y1:y2,x1:x2,c]) + (alpha_png*png_img[yy1:yy2,xx1:xx2,c]))
    # 4通道转换成3通道
    jpg_img = jpg_img[:,:,:3]
 
    return jpg_img

In [4]:
import face_recognition
import cv2
import numpy as np
import pinyin.cedict

# This is a super simple (but slow) example of running face recognition on live video from your webcam.
# There's a second example that's a little more complicated but runs faster.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

known_path = "C:/Users/14640/Pictures/known_pictures/"
imgnames = os.listdir(known_path)
print(imgnames)

known_images = []
known_face_encodings = []
known_face_names = []

for i in range(len(imgnames)):
    print(i)
    print(known_path+imgnames[i])
    
    # Load a sample picture and learn how to recognize it.
    known_img = face_recognition.load_image_file(known_path+imgnames[i])
    known_face_encoding = face_recognition.face_encodings(known_img)
    
    # 如果不能检测到脸
    if known_face_encoding == []:
        print(known_path+imgnames[i]+' ---->>> 没有检测到人脸')
        
    known_face_encodings.append(known_face_encoding[0])
    known_images.append(known_img)
    
    # 获取成中文名
    china_name = os.path.splitext(imgnames[i])[0]
    print(china_name)
    
    # 获取拼音
    pinyin_name = pinyin.get(china_name, format="strip", delimiter=" ")
    print(pinyin_name)
    
    known_face_names.append(pinyin_name)
    
# # Load a second sample picture and learn how to recognize it.
# biden_image = face_recognition.load_image_file("biden.jpg")
# biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
# known_face_encodings

# known_face_names = [
# #     "Joe Biden"
# ]

while True:
    # 边框(透明png)
    bod = cv2.imread(r'C:\Users\14640\Pictures\bodimg\ds.png',-1)
    
    # Grab a single frame of video
    ret, frame = video_capture.read()
    
    fh,fw,fs = frame.shape

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame, number_of_times_to_upsample=1, model='hog')
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations, num_jitters=1, model='large')

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        # if True in matches:
        #     first_match_index = matches.index(True)
        #     name = known_face_names[first_match_index]

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

            
        # Draw a box around the face
#         cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255), 2)
        
    
        # 添加 png(bod边框)
        if top-40 >= 0:
            fac_t = top-40
        else:
            fac_t = 0
        if bottom+10 <= fh:
            fac_b = bottom+10
        else:
            fac_b = fh
        if left-10 >= 0:
            fac_l = left-10
        else:
            fac_l = 0
        if right+10 <= fw:
            fac_r = right+10
        else:
            fac_r = fw
        fac = frame[fac_t: fac_b, fac_l: fac_r]  # 第top行到第bottom行，第left列到第right列
        h,w,s=fac.shape
        bod=cv2.resize(bod,(w,h),interpolation=cv2.INTER_LINEAR)#两图像补充为一样的大小
        fac=merge_img(fac, bod, 0, h, 0, w)
        frame[fac_t: fac_b, fac_l: fac_r] = fac

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255,255,255), 1)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + int(w*0.15), bottom - int(h*0.1)), font, 0.5, (255,191,0), 1)
        cv2.putText(frame, str(np.min(face_distances))[0:6], (left + int(w*0.2), bottom + int(h*0.1)), font, 0.5, (255,191,0), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

['刘青沅.jpg', '宋佳乐.jpg', '张泰.jpg', '菅韧.jpg']
0
C:/Users/14640/Pictures/known_pictures/刘青沅.jpg
刘青沅
liu qing yuan
1
C:/Users/14640/Pictures/known_pictures/宋佳乐.jpg
宋佳乐
song jia le
2
C:/Users/14640/Pictures/known_pictures/张泰.jpg
张泰
zhang tai
3
C:/Users/14640/Pictures/known_pictures/菅韧.jpg
菅韧
jian ren


In [31]:
bod = cv2.imread(r'C:\Users\14640\Pictures\bodimg\ds.png',-1)
cv2.imshow('bod', bod)
cv2.waitKey(0)
cv2.destroyAllWindows()